In [ ]:
raw_fold = pathlib.Path('H:/UGS_Flux/Data_Downloads/Wellington')
value = 'm20240723'
raw_met_data = micromet.raw_file_compile(raw_fold, value, search_str = "*AmeriFlux*.dat")

print(pd.to_datetime(raw_met_data['TIMESTAMP_START'][0],format='%Y%m%d%H%M'))

raw_met_data['datetime'] = pd.date_range(start='2024-07-11 06:00:00', freq='30min', periods=len(raw_met_data))
raw_met_data.set_index('datetime', inplace=True)
#micromet.Reformatter(raw_met_data)

In [ ]:
data.rename(columns={'WD':'csat_WD'})['csat_WD']

In [ ]:
df_wd = pd.concat([data.rename(columns={'WD':'csat_WD','WS':'csat_WS'}),
                  raw_met_data.rename(columns={'WD':'young_WD','WS':'young_WS'}).shift(-1)], axis=1)
df_wd = df_wd[['csat_WD','young_WD','csat_WS','young_WS']].dropna(axis=1, how='all')
df_wd['csat_wd_rad'] = pg.circ_axial(pg.convert_angles(df_wd['csat_WD'],positive=True),2)
csat_wd_mean = pg.circ_mean(pg.convert_angles(df_wd['csat_WD']))
df_wd['young_wd_rad'] = pg.circ_axial(pg.convert_angles(df_wd['young_WD'],positive=True),2)
young_wd_mean = pg.circ_mean(pg.convert_angles(df_wd['young_WD']))



print(csat_wd_mean, young_wd_mean)
pg.circ_corrcc(df_wd['csat_wd_rad'], df_wd['young_wd_rad'], correction_uniform=True)

In [ ]:
df_wd.loc[(df_wd['young_wd_rad']>3.2) & (df_wd['csat_wd_rad']<2),'csat_wd_rad'] = df_wd.loc[(df_wd['young_wd_rad']<3.2) & (df_wd['csat_wd_rad']>2),'csat_wd_rad'] + 6.25

fig = px.scatter(df_wd, x='csat_wd_rad', y='young_wd_rad', trendline="ols",color=df_wd.index.hour)
fig.show()

In [ ]:

#fig = px.line(df_wd, y=["young_WS","csat_WS"],
#              title='custom tick labels')
fig = px.scatter(df_wd, x='young_WS', y='csat_WS', trendline="ols",color=df_wd.index.hour)
fig.show()

In [ ]:
#raw_met_data['SW_IN'].plot()

comp_col = 'LW_OUT'

sql = "SELECT * FROM groundwater.amfluxeddy WHERE (groundwater.amfluxeddy.stationid = 'US-UTW') AND (groundwater.amfluxeddy.timestamp_start >= 202407110000)"
data = pd.read_sql(sql, con=engine)
data = data.replace(-9999, np.nan)
data.set_index('datetime_start', inplace=True)
for col in data.columns:
    data.rename(columns={col:col.upper()}, inplace=True)
#data['SW_IN'].plot()

df = pd.concat([data.rename(columns={f'{comp_col}':f'{comp_col}_HF'})[f'{comp_col}_HF'].shift(-1),
                raw_met_data.rename(columns={f'{comp_col}':f'{comp_col}_AP'})[f'{comp_col}_AP']],axis=1).dropna()


y = df[f'{comp_col}_HF']
x = df[f'{comp_col}_AP']
X = sm.add_constant(x)
model = sm.OLS(y,X)
results = model.fit()
print(results.params)



fig = px.scatter(df, x=f'{comp_col}_HF', y=f'{comp_col}_AP', trendline="ols",color=df.index.hour)

ax = pg.plot_blandaltman(x, y)

fig.show()
print(results.summary())

In [ ]:
d = pg.plot_blandaltman(x, y)